### **LLMs as re-rankers**

This notebook compares single vector retriever with vector retrieval + reranking (using gpt 3.5 and gpt 4)

**Paper**: Is ChatGPT Good at Search? Investigating Large Language Models as Re-Ranking Agents - https://arxiv.org/abs/2304.09542  

**The test**: **How effective are LLMs are at re-ranking to identify target sentence to answer a question?**

I test out the use of LLMs as rerankers (from https://github.com/sunnweiwei/RankGPT) by injecting random sentences into a text document (1 target sentence and 8 distractors). Sentences are inserted at random positions in paul graham's essays, spaced at 256 words apart. The text documents are then split using langchain recursive character splitter with chunk size as 256. This ensures that each chunk doesn't have two sentences injected into them. The posiiton of the chunk with the target sentence (target chunk) is then compared along with the time taken for retrieval. 

**Question:**
- "Was paul vegan?"

**Target sentence:**
- "Paul liked to eat McChicken burger."

**Example of distractors:**
- "Paul met McChicken at the library, where they discussed their favorite novels."  
- "While at the concert, Paul noticed Charlie enjoying a McChicken during the intermission, savoring each bite as he discussed the finer points of music."

**Retrieval Methods**:
- vanilla retrieval (single vector text-ada) of 10 chunks
- vanilla retrieval of 10 chunks + gpt 3.5 turbo 
- vanilla retrieval of 10 chunks + gpt 4 turbo 

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()  
import warnings
import random
warnings.filterwarnings('ignore')
api_key = os.environ.get("OPENAI_API_KEY")


In [6]:
def load_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        return "The file was not found."
    except Exception as e:
        return f"An error occurred: {e}"


In [7]:
def convert_documents_to_query_structure(documents, query, method, duration):
    return {
        'query': query,
        'hits': [{'content': doc.page_content} for doc in documents],
        "method": method, # Add your custom key-value pair here
        "duration": duration
    }

**Functions to inject random sentence**

In [8]:
def find_injection_position(words, start_position):
    # Generate a list of potential positions (after a period) for injection
    potential_positions = [i + 1 for i in range(start_position, len(words)) if '.' in words[i]]

    # If there are suitable positions found, return a random one
    if potential_positions:
        # Ensure the selected position is at least 256 words away from the start_position
        filtered_positions = [pos for pos in potential_positions if pos > start_position + 256]
        if filtered_positions:
            return random.choice(filtered_positions)

    # If no suitable position is found or if they are too close, return the end of the text
    return len(words)


In [9]:
def inject_sentences(text, sentences_to_inject):
    # Split text into words
    original_words = text.split()

    # Shuffle the list of sentences to inject
    random.shuffle(sentences_to_inject)

    # The word count where the next sentence can be injected
    next_inject_position = 0

    # Iterate over the shuffled sentences to inject
    for sentence in sentences_to_inject:
        # Find a random position to inject this sentence
        # Ensure it's at least 256 words apart from the last injection
        position = find_injection_position(original_words, next_inject_position)

        # Prepare connecting words
        connecting_words = ["Furthermore, ", "Additionally, ", "On another note, ", "Moreover, "]

        # Choose a random connecting word
        connector = random.choice(connecting_words)

        # Inject the sentence
        original_words.insert(position, connector + sentence)

        # Update the next injection position
        next_inject_position = position + len(sentence.split()) + 256

    # Reassemble the text
    modified_text = ' '.join(original_words)
    return modified_text


**List of sentences to inject**

In [10]:
sentences_to_inject = sentences = [
    "Paul liked to eat McDonald's McChicken burger.",
    "Paul met McChicken at the library, where they discussed their favorite novels.",
    "In the park, Paul played chess with McChicken, who turned out to be a formidable opponent.",
    "During the community meeting, Paul and Burgers collaborated on a plan to beautify the neighborhood.",
    "McChicken and Paul volunteered together at the local animal shelter, taking care of abandoned pets.",
    "While at the concert, Paul noticed Burgers enjoying a McChicken during the intermission, savoring each bite as he discussed the finer points of music.",
    "During their morning jog, Paul saw McChicken energetically biting into a McChicken sandwich, talking about the balance of enjoying life while staying fit.",
    "In the pottery class, amidst the clay and creativity, Paul watched as McChicken took a thoughtful bite of a McChicken, reflecting on their shared passion for the arts.",
    "As they celebrated their first-place win in the science competition, Paul observed Burgers treating himself to a well-deserved McChicken, their success making it taste even better.",
    "At the museum, amidst a discussion of world history, Paul saw McChicken passionately explain the significance of ancient events, pausing only to enjoy bites of his McChicken sandwich."
]


**The Test**

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from rank_gpt import permutation_pipeline
import time

Inject given random sentences into text

In [12]:
def prep_text(file_path, sentences_to_inject):
    text = load_text_file(file_path)
    injected_text = inject_sentences(text, sentences_to_inject)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=16,
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.create_documents([injected_text])
    return texts

Perform vanilla retrieval or vanilla retrieval + reranking

In [13]:
def perform_retrieval(method, texts, api_key):
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever(search_kwargs={"k": 20})
    query = "was paul vegan?"

    start_time = time.time()
    retrievals = retriever.get_relevant_documents(query)
    items = convert_documents_to_query_structure(retrievals, query, method, 0)

    if method != "vanilla":
        new_items = permutation_pipeline(items, rank_start=0, rank_end=19, model_name=method, api_key=api_key)
        new_items['duration'] = time.time() - start_time
        return new_items
    else:
        items['duration'] = time.time() - start_time
        return items

In [14]:
file_path = './docs/paul_graham_essay.txt'
documents = []
methods = ["vanilla", "gpt-3.5-turbo-1106", "gpt-4-1106-preview"]
n = 1

Run loop - n times for each method

In [15]:
for method in methods:
    for i in range(0, n):
        injected_texts = prep_text(file_path, sentences_to_inject)
        result = perform_retrieval(method, injected_texts, api_key)
        documents.append(result)


/Users/arjun/Documents/github/rag-experiments/ragnv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Extract df from results

In [23]:
import pandas as pd

def find_target_sentence_in_documents(documents, target_sentence):
    # Initialize an empty list to collect rows
    rows = []

    # Iterate through the documents
    for doc in documents:
        hits = doc['hits']
        method = doc.get('method')  # Use get() to avoid KeyError if 'method' does not exist
        duration = doc.get('duration')
        
        for index, item in enumerate(hits):
            content = item.get('content')  # Use get() to avoid KeyError if 'content' does not exist
            if target_sentence in content:
                # Create a dictionary for the row and add to the list
                rows.append({'position': index + 1, 'method': method, 'duration': duration})
                break  # Exit the loop once the target sentence is found

    # Create a DataFrame from the list of rows
    df = pd.DataFrame(rows)

    return df

# Example usage
target_sentence = "Paul liked to eat McDonald's McChicken burger."
df = find_target_sentence_in_documents(documents, target_sentence)

In [24]:
df.head()

,position,method,duration
0,2,vanilla,1.408767
1,1,gpt-3.5-turbo-1106,1.721095
2,1,gpt-4-1106-preview,3.321413


Visualing target chunk position

In [25]:
# Define colors for each method
color_map = {
    'vanilla': '#2ca02c',
    'gpt-3.5-turbo-1106': '#1f77b4',
    'gpt-4-1106-preview': '#d62728'
}


In [40]:
import pandas as pd
import plotly.express as px


# Prepare data for plotting
df_count = df.groupby(['position', 'method']).size().reset_index(name='count')

# Create a bar plot using Plotly
fig = px.bar(df_count, x='position', y='count', color='method', barmode='group',
             title='Target chunk position from 10 runs',
             color_discrete_map=color_map)

# Customize axis labels, layout, and apply dark theme
fig.update_xaxes(title_text='Position')
fig.update_yaxes(title_text='Count', tickformat='d')
fig.update_layout(
    template='plotly_dark',
    legend_title_text='Method',
    bargap=0.1,  # Smaller gap between bars of different x-coordinates
    bargroupgap=0.1  # Smaller gap within groups of bars
)

# Update legend names
legend_names = {'vanilla': 'vanilla retrieval', 'gpt-3.5-turbo-1106': 'vanilla retrieval + gpt-3.5 reranking', 'gpt-4-1106-preview': 'vanilla retrieval + gpt-4 reranking'}
for trace in fig.data:
    trace.name = legend_names.get(trace.name, trace.name)

# Show the plot
fig.show()


Plot of average duration

In [39]:
import pandas as pd
import plotly.express as px


# Create a bar plot for duration with specified colors
fig = px.bar(df, x='method', y='duration', color='method',
             title='Average duration across 10 runs',
             color_discrete_map=color_map)

# Customize axis labels and apply dark theme
fig.update_xaxes(title_text='Method')
fig.update_yaxes(title_text='Duration (seconds)')
fig.update_layout(template='plotly_dark')

# Make the bars thinner
fig.update_traces(marker=dict(line=dict(width=1)))

# Update legend names
legend_names = {'vanilla': 'vanilla retrieval', 'gpt-3.5-turbo-1106': 'vanilla retrieval + gpt-3.5 reranking', 'gpt-4-1106-preview': 'vanilla retrieval + gpt-4 reranking'}
for trace in fig.data:
    trace.name = legend_names.get(trace.name, trace.name)


# Show the plot
fig.show()
